# Algoritmos paralelos: Pensando en paralelo
Ahora que comenzamos a ver los desafíos del paralelismo, vale la pena dar un paso atrás y examinar cómo podemos diseñar algoritmos paralelos.
Esta es una adaptación de [workshop paper](http://jiahao.github.io/parallel-prefix/) por Jiahao Chen y Alan Edelman titulado  "*Parallel Prefix Polymorphism Permits Parallelization, Presentation '&' Proof*" y aparecerá en los eventos de [First Workshop for High Performance Technical Computing in Dynamic
Languages](http://jiahao.github.io/hptcdl-sc14/) celebrada en conjunto con [SC14: The International Conference on High Performance Computing, Networking, Storage and Analysis](http://sc14.supercomputing.org/)


In [4]:
]add Pkg

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
   Installed Glib_jll ───── v2.74.0+2
   Installed Libiconv_jll ─ v1.16.1+2
    Updating `~/.julia/environments/v1.8/Project.toml`
  [44cfe95a] + Pkg v1.8.0
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [7746bdde] ↑ Glib_jll v2.74.0+1 ⇒ v2.74.0+2
  [94ce4f54] ↑ Libiconv_jll v1.16.1+1 ⇒ v1.16.1+2
Precompiling project...
  ✓ JuliaAcademyData
  ✓ Libiconv_jll
  ✓ XML2_jll
  ✓ Gettext_jll
  ✓ XSLT_jll
  ✓ Wayland_jll
  ✓ Glib_jll
  ✓ Xorg_libxcb_jll
  ✓ Xorg_xcb_util_jll
  ✓ Xorg_libX11_jll
  ✓ Xorg_xcb_util_image_jll
  ✓ Xorg_xcb_util_renderutil_jll
  ✓ Xorg_xcb_util_keysyms_jll
  ✓ Xorg_xcb_util_wm_jll
  ✓ Xorg_libxkbfile_jll
  ✓ Xorg_libXfixes_jll
  ✓ Xorg_libXrender_jll
  ✓ Xorg_libXext_jll
  ✓ gdk_pixbuf_jll
  ✓ Libglvnd_jll
  ✓ Xorg_libXinerama_jll
  ✓ Xorg_libXi_jll
  ✓ Xorg_xkbcomp_jll
  ✓ Xorg_libXcursor_jll
  ✓ 

In [7]:
import Pkg

In [8]:
Pkg.add("Compose")

   Resolving package versions...
   Installed Compose ─ v0.9.4
    Updating `~/.julia/environments/v1.8/Project.toml`
  [a81c6b42] + Compose v0.9.4
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [a81c6b42] + Compose v0.9.4
Precompiling project...
  ✓ Compose
  1 dependency successfully precompiled in 2 seconds. 281 already precompiled. 1 skipped during auto due to previous errors.


In [10]:
Pkg.add("Gadfly")

   Resolving package versions...
   Installed StatsFuns ────────── v1.1.1
   Installed PlotUtils ────────── v1.3.2
   Installed TranscodingStreams ─ v0.9.10
   Installed HTTP ─────────────── v1.6.1
   Installed Parsers ──────────── v2.5.2
   Installed Hexagons ─────────── v0.2.0
   Installed CategoricalArrays ── v0.10.7
   Installed ExproniconLite ───── v0.7.9
   Installed Plots ────────────── v1.37.2
   Installed Colors ───────────── v0.12.9
   Installed GPUArraysCore ────── v0.1.2
   Installed CoupledFields ────── v0.2.0
   Installed StructArrays ─────── v0.6.14
   Installed FillArrays ───────── v0.13.6
   Installed Gadfly ───────────── v1.3.4
   Installed Interpolations ───── v0.14.7
   Installed PrettyTables ─────── v2.2.2
   Installed Luxor ────────────── v3.6.0
    Updating `~/.julia/environments/v1.8/Project.toml`
  [c91e804a] + Gadfly v1.3.4
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [324d7699] + CategoricalArrays v0.10.7
  [5ae59095] ↑ Colors v0.12.8 ⇒ v0.12.9
⌅

In [11]:
using Compose, Gadfly

┌ Info: Precompiling Gadfly [c91e804a-d5a3-530f-b6f0-dfbca275c004]
└ @ Base loading.jl:1664
┌ Warning: Module Compose with build ID 4344716308180 is missing from the cache.
│ This may mean Compose [a81c6b42-2e10-5240-aca2-a61377ecd94b] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1325
┌ Info: Skipping precompilation since __precompile__(false). Importing Gadfly [c91e804a-d5a3-530f-b6f0-dfbca275c004].
└ @ Base loading.jl:1341


# `reduce()`

La reducción aplica un operador binario a un vector repetidamente para devolver un escalar. Así, + se convierte en suma y * se convierte en producto.

Se considera una primitiva de computación paralela básica.

In [12]:
reduce(+, 1:8), sum(1:8)  # triangular numbers

(36, 36)

In [13]:
reduce(*, 1:8), prod(1:8) # factorials

(40320, 40320)

In [15]:
boring(a,b)=a
@show reduce(boring, 1:8)
boring2(a,b)=b
@show reduce(boring2, 1:8)

reduce(boring, 1:8) = 1
reduce(boring2, 1:8) = 8


8

    Tambien puedes usar reduce() para calcular los numeros de fibonacci usando sus recurrencias.
$$\begin{pmatrix} f_2 \\f_1 \end{pmatrix} = \begin{pmatrix} f_1 + f_0 \\ f_1 \end{pmatrix}
= \begin{pmatrix} 1 & 1 \\ 1 & 0 \end{pmatrix} \begin{pmatrix} f_1 \\ f_0 \end{pmatrix} $$

$$\begin{pmatrix} f_{n+1} \\ f_n \end{pmatrix} = \dots
= \begin{pmatrix} 1 & 1 \\ 1 & 0 \end{pmatrix}^n \begin{pmatrix} f_1 \\ f_0 \end{pmatrix} $$

A partir de esto, puedes demostrar que
$$\begin{pmatrix} 1 & 1 \\ 1 & 0 \end{pmatrix}^n  = \begin{pmatrix} f_{n+1} & f_n \\ f_n & f_{n-1} \end{pmatrix} $$

(esto se aplica reducir al mismo argumento una y otra vez; por supuesto, hay otras formas)

In [57]:
M=[1 1; 1 0]
reduce(*,fill(M,3))
prod(fill(M,3))

2×2 Matrix{Int64}:
 3  2
 2  1

In [62]:
n= 10 #Intente manipular n para elegir diferentes valores (pruebe entre 0 y 100)
prod(fill(big.(M),n))

2×2 Matrix{BigInt}:
 89  55
 55  34

In [63]:
fib(j)=reduce(*, fill(M,j))
fib.([4,7])

2-element Vector{Matrix{Int64}}:
 [5 3; 3 2]
 [21 13; 13 8]

Puede resolver recurrencias de cualquier complejidad usando `reduce()`. Por ejemplo, `reduce()` puede calcular una matriz de Hadamard a partir de su definición en términos de sus submatrices:

$$H_2 = \begin{pmatrix} H_1 & H_1 \\ H_1 & -H_1 \end{pmatrix} = \begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix} \otimes H_1$$

y así.

(Nota: esto es solo usar reduce para calcular una matriz de potencia.
Uno puede pensar en formas alternativas con seguridad.)

In [64]:
# Si A es m x n
# Si B es p x q
# entonces kron(A,B) es mp x nq y tiene todos los elementos de A por todos los elementos de B

In [65]:
A=[1 2;3 4]
B=[10 100; 1 -10]
⊗(A,B)=kron(A,B)

M=[ 1 1;1 -1]
H=⊗(⊗(⊗(M,M),M),M)

16×16 Matrix{Int64}:
 1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 1  -1   1  -1   1  -1   1  -1   1  -1   1  -1   1  -1   1  -1
 1   1  -1  -1   1   1  -1  -1   1   1  -1  -1   1   1  -1  -1
 1  -1  -1   1   1  -1  -1   1   1  -1  -1   1   1  -1  -1   1
 1   1   1   1  -1  -1  -1  -1   1   1   1   1  -1  -1  -1  -1
 1  -1   1  -1  -1   1  -1   1   1  -1   1  -1  -1   1  -1   1
 1   1  -1  -1  -1  -1   1   1   1   1  -1  -1  -1  -1   1   1
 1  -1  -1   1  -1   1   1  -1   1  -1  -1   1  -1   1   1  -1
 1   1   1   1   1   1   1   1  -1  -1  -1  -1  -1  -1  -1  -1
 1  -1   1  -1   1  -1   1  -1  -1   1  -1   1  -1   1  -1   1
 1   1  -1  -1   1   1  -1  -1  -1  -1   1   1  -1  -1   1   1
 1  -1  -1   1   1  -1  -1   1  -1   1   1  -1  -1   1   1  -1
 1   1   1   1  -1  -1  -1  -1  -1  -1  -1  -1   1   1   1   1
 1  -1   1  -1  -1   1  -1   1  -1   1  -1   1   1  -1   1  -1
 1   1  -1  -1  -1  -1   1   1  -1  -1   1   1   1   1  -1  -1
 1  -1  -1   1  -1   1   1  -1  -1

In [66]:
H'H

16×16 Matrix{Int64}:
 16   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0  16   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0  16   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0  16   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0  16   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0  16   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0  16   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0  16   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0  16   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0  16   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0  16   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0  16   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0  16   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0  16   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0  16   0
  0   0   0   0   0

In [67]:
Hadamard(n)=reduce(⊗, fill(M,n))
H=Hadamard(4)

16×16 Matrix{Int64}:
 1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 1  -1   1  -1   1  -1   1  -1   1  -1   1  -1   1  -1   1  -1
 1   1  -1  -1   1   1  -1  -1   1   1  -1  -1   1   1  -1  -1
 1  -1  -1   1   1  -1  -1   1   1  -1  -1   1   1  -1  -1   1
 1   1   1   1  -1  -1  -1  -1   1   1   1   1  -1  -1  -1  -1
 1  -1   1  -1  -1   1  -1   1   1  -1   1  -1  -1   1  -1   1
 1   1  -1  -1  -1  -1   1   1   1   1  -1  -1  -1  -1   1   1
 1  -1  -1   1  -1   1   1  -1   1  -1  -1   1  -1   1   1  -1
 1   1   1   1   1   1   1   1  -1  -1  -1  -1  -1  -1  -1  -1
 1  -1   1  -1   1  -1   1  -1  -1   1  -1   1  -1   1  -1   1
 1   1  -1  -1   1   1  -1  -1  -1  -1   1   1  -1  -1   1   1
 1  -1  -1   1   1  -1  -1   1  -1   1   1  -1  -1   1   1  -1
 1   1   1   1  -1  -1  -1  -1  -1  -1  -1  -1   1   1   1   1
 1  -1   1  -1  -1   1  -1   1  -1   1  -1   1   1  -1   1  -1
 1   1  -1  -1  -1  -1   1   1  -1  -1   1   1   1   1  -1  -1
 1  -1  -1   1  -1   1   1  -1  -1

In [69]:
using LinearAlgebra
cumsum(1:8)  # Es útil saber que cumsum es un operador lineal
# ¡Puedes usar el metodo power! A continuación se muestra la matriz subyacente.
A=tril(ones(Int,8,8))

8×8 Matrix{Int64}:
 1  0  0  0  0  0  0  0
 1  1  0  0  0  0  0  0
 1  1  1  0  0  0  0  0
 1  1  1  1  0  0  0  0
 1  1  1  1  1  0  0  0
 1  1  1  1  1  1  0  0
 1  1  1  1  1  1  1  0
 1  1  1  1  1  1  1  1

In [72]:
[A*(1:8),cumsum(1:8)]

2-element Vector{Vector{Int64}}:
 [1, 3, 6, 10, 15, 21, 28, 36]
 [1, 3, 6, 10, 15, 21, 28, 36]

# `prefix`

Having discussed `reduce`, we are now ready for the idea behind prefix sum.
Prefix or scan is long considered an important parallel
primitive as well.

Habiendo discutido `reduce`, ahora estamos listos para la idea detrás del prefijo sum.
El prefijo o escaneo se considera durante mucho tiempo un paralelo importante
primitivo también.

Suponga que queria calcular las sumas parciales de un vector, es decir, dado`y[1:n]`, queremos sobre escribir `y` con el vector de sumas parciales

```julia
new_y[1] = y[1]
new_y[2] = y[1] + y[2]
new_y[3] = y[1] + y[2] + y[3]
...
```
A primera vista, parece imposible paralelizar esto, ya que

```julia
new_y[1] = y[1]
new_y[2] = new_y[1] + y[2]
new_y[3] = new_y[2] + y[3]
...
```
que parece ser un proceso intrínsecamente serial. Como se escribe con un `+`
operador, esto es `cumsum`, pero tenga en cuenta que puede generalizarse a cualquier operación.

In [73]:
function prefix_serial!(y, ⊕)
    for i=2:length(y)
        y[i] = y[i-1] ⊕ y[i]
    end
    y
end

prefix_serial! (generic function with 1 method)

In [74]:
prefix_serial!([1:8;],+)

8-element Vector{Int64}:
  1
  3
  6
 10
 15
 21
 28
 36

In [75]:
cumsum(1:8)

8-element Vector{Int64}:
  1
  3
  6
 10
 15
 21
 28
 36

In [76]:
prefix_serial!([1:8;], *)

8-element Vector{Int64}:
     1
     2
     6
    24
   120
   720
  5040
 40320

In [77]:
cumprod(1:8)

8-element Vector{Int64}:
     1
     2
     6
    24
   120
   720
  5040
 40320

Sin embargo, resulta que debido a que estas operaciones son asociativas, podemos reagrupar el orden de cómo se realizan estas sumas o productos. (Esto, por supuesto, también se extiende a otras operaciones asociativas). ¡Prefix8! proporciona otra ordenación de 8 operaciones asociativas:

In [78]:
# Magic :)
function prefix8!(y, ⊕)
    length(y)==8 || error("length 8 only")
    for i in (2,4,6,8); y[i] = y[i-1] ⊕ y[i]; end
    for i in (  4,  8); y[i] = y[i-2] ⊕ y[i]; end
    for i in (      8); y[i] = y[i-4] ⊕ y[i]; end
    for i in (    6  ); y[i] = y[i-2] ⊕ y[i]; end
    for i in ( 3,5,7 ); y[i] = y[i-1] ⊕ y[i]; end
    y
end

prefix8! (generic function with 1 method)

In [79]:
prefix8!([1:8;], +) == cumsum(1:8)

true

De hecho, esto puede generalizarse más allá de las matrices de longitud 8:

In [81]:
# More magic
function prefix!(y, ⊕)
    l=length(y)
    k=ceil(Int, log2(l))
    @inbounds for j=1:k, i=2^j:2^j:min(l, 2^k)              #"reduce"
        y[i] = y[i-2^(j-1)] ⊕ y[i]
    end
    @inbounds for j=(k-1):-1:1, i=3*2^(j-1):2^j:min(l, 2^k) #"expand"
        y[i] = y[i-2^(j-1)] ⊕ y[i]
    end
    y
end

prefix! (generic function with 1 method)

In [82]:
A = rand(0:9, 123)
prefix!(copy(A), *) == cumprod(A)

true

## ¿Qué es esta magia?

We can visualize the operations with a little bit of trickery. In Julia, arrays are simply types that expose the array protocol. In particular, they need to implement  methods for the generic functions `length`, `getindex` and `setindex!`. The last two are used in indexing operations, since statements

Podemos visualizar las operaciones con un poco de engaño. En Julia, las matrices son simplemente tipos que exponen el protocolo de matriz. En particular, necesitan implementar métodos para las funciones genéricas `length`, `getindex` y `setindex!`. Los dos últimos se utilizan en operaciones de *indexación*, ya que las sentencias

    y = A[1]
    A[3] = y
